Homework 5

1. Compile a list of static links (permalinks) to individual user movie reviews from one particular
website. This will be your working dataset for this assignment, as well as for assignments 7 and
8, which together will make up your semester project.
a. It does not matter if you use a crawler or if you manually collect the links, but you will
need at least 100 movie review links. Note that, as of this writing, the robots.txt file of
IMDB.com allows the crawling of user reviews.
b. Each link should be to a web page that has only one user review of only one movie, e.g.,
the user review permalinks on the IMDB site.
c. Choose reviews of movies that are all in the same genre, e.g., sci-fi, mystery, romance,
superhero, etc.
d. Make sure your collection includes reviews of several movies in your chosen genre and
that it includes a mix of negative and positive reviews.





In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [155]:
import nltk
from nltk.stem import PorterStemmer
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
nltk.download("punkt")
from nltk import word_tokenize
nltk.download('universal_tagset')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [156]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [160]:
import spacy
sp = spacy.load('en_core_web_sm')

In [251]:
url=["https://www.rottentomatoes.com/m/arrival_2016/reviews",
     "https://www.rottentomatoes.com/m/guardians_of_the_galaxy/reviews",
     "https://www.rottentomatoes.com/m/guardians_of_the_galaxy_vol_2/reviews",
     "https://www.rottentomatoes.com/m/star_wars_episode_vii_the_force_awakens/reviews",
     "https://www.rottentomatoes.com/m/interstellar_2014/reviews"]

title=["Arrival",
     "guardians_of_the_galaxy",
     "guardians_of_the_galaxy_vol_2",
     "star_wars_episode_vii_the_force_awakens",
     "interstellar_2014"]


In [253]:
i=0

review_text_raw = []
url_raw = []
author_raw = []
title_raw = []

while i < 5:
  response = requests.get(url[i])
  content = response.text
  z=0
  # Create a BeautifulSoup object
  soup = BeautifulSoup(content, "html.parser")

  # Find all the review containers
  review_containers = soup.find_all("div", class_="review-row")

  # Iterate over each review container
  for container in review_containers:
      # Extract review text
      review_text = container.find("p", class_="review-text").text.strip()
      review_text_raw.append(review_text)
      print(review_text)

      # Extract review url
      review_url = container.find("a", {'class': 'full-url'}, href=True)
      url_raw.append(review_url)

      #Extract reviewer name
      reviewer = container.find("a", class_="display-name").text.strip()
      author_raw.append(reviewer)

      title_raw.append(title[i])

      # Extract review URL
    # review_url = container.find("a", class_="full-url")["href"].strip()
    # print(review_text)
      print(z)
      z+=1
  i+=1

Leaves us with hope... One of the best sci-fi films of recent times without gunfights. [Full review in Spanish]
0
Melancholy, mysterious and measured as it tracks the struggle to communicate with these alien creatures, Arrival is really about language and how it functions, how it shapes us and sometimes limits us, and how it can connect or separate us.
1
Amy Adams’ transcendent portrayal of a linguist...
2
I appreciated the intelligent science fiction, but also how the film steps beyond genre. It turned out to be far more intimate and thought-provoking than I ever expected. And all of that on top of the superb visuals, art direction, and score.
3
The trippy events unfolding out of Heisserer’s screenplay tangle the puppeteer’s strings and play with narrative and filmmaking forces few are daring enough, and smart enough, to wield.
4
Both cerebral and achingly emotional, Arrival sustains a message about hope and understanding for a better humanity that audiences may need right now.
5
One 

In [256]:
df = pd.DataFrame({'Title': list(title_raw), 'Author': list(author_raw), 'Review':list(review_text_raw), 'Url':list(url_raw)}, columns=['Title','Author', 'Review', 'Url'])
df['Review'] = df['Review'].map(lambda x: x.lstrip('+-').rstrip('[Full review in Spanish]'))
#df['Url'] = df['Url'].map(lambda x: x.lstrip('+-').rstrip('<a class='))
#df['Url'] = df['Url'].map(lambda x: x.lstrip('+-').rstrip('" rel="nofollow noopener" target="_blank">Full Review</a>'))

df

,Title,Author,Review,Url
0,Arrival,Erick Estrada,Leaves us with hope... One of the best sci-fi ...,[Full Review]
1,Arrival,Alison Gillmor,"Melancholy, mysterious and measured as it trac...",[Full Review]
2,Arrival,Brian T. Carney,Amy Adams’ transcendent portrayal of a linguis...,[Full Review]
3,Arrival,Keith Garlington,"I appreciated the intelligent science fiction,...",[Full Review]
4,Arrival,Don Shanahan,The trippy events unfolding out of Heisserer’s...,[Full Review]
...,...,...,...,...
95,interstellar_2014,Stephen A. Russell,None of these characters feel fully-fledged......,[Full Review]
96,interstellar_2014,Kelechi Ehenulo,Interstellar is not Christopher Nolan's best f...,[Full Review]
97,interstellar_2014,Brent McKnight,As spectacular as it is flawed.,[Full Review]
98,interstellar_2014,Damond Fudge,Interstellar is not only a grand space adventu...,[Full Review]


2. Extract noun phrase (NP) chunks from your reviews using the following procedure:
a. In Python, use BeautifulSoup to grab the main review text from each link.
b. Next run each review text through a tokenizer, and then try to NP-chunk it with a
shallow parser.
c. You probably will have too many unknown words, owing to proper names of characters,
actors, and so on that are not in your working dictionary. Make sure the main names
that are relevant to the movies in your collection of reviews are added to the working
lexicon, and then run the NP chunker again.


In [245]:
df['Review'][i]

'One of the most beautiful, emotional and original science-fiction films of the last decade.'

In [274]:
# New stop words list
customize_stop_words = [
    'Nolan', 'Arrival', 'Guardians of Galaxy', 'Arrival', 'Star Wars'
]

# Mark them as stop words
for w in customize_stop_words:
    nlp.vocab[w].is_stop = True

In [275]:
nlp = spacy.load('en_core_web_sm')
title_for_np = []
i=0

np_chunk = []
while i < len(df):
  df['Review'] = df.Review.apply(lambda text:
                                          " ".join(token.lemma_ for token in nlp(text)
                                                   if not token.is_stop))
  doc = nlp(df['Review'][i])
  for noun_chunk in doc.noun_chunks:
    np_chunk.append(noun_chunk)
    title_for_np.append(df['Title'][i])
    print( df['Title'][i], noun_chunk)

  i+=1

Arrival hope
Arrival good sci - fi film recent time gunfight
Arrival melancholy , mysterious measure track struggle
Arrival alien creature
Arrival Arrival language function
Arrival shape limit
Arrival Amy Adams ’ transcendent portrayal linguist
Arrival intelligent science fiction
Arrival far intimate thought - provoke
Arrival art direction
Arrival trippy event
Arrival Heisserer screenplay tangle puppeteer string
Arrival narrative filmmake force
Arrival Arrival
Arrival message hope
Arrival humanity audience
Arrival beautiful , emotional original science - fiction film decade
Arrival good film
Arrival motion picture
Arrival far focused humanity
Arrival transcendent view experience skin heart
Arrival arrival
Arrival sci - fi story
Arrival hypnotic strange beautiful - beautiful time
Arrival word
Arrival astonishingly beautiful film
Arrival fine , fine , science fiction movie
Arrival 21st Century
Arrival true theme
Arrival Arrival
Arrival premonition tangible event
Arrival slow - burn thril

In [276]:
np_df = pd.DataFrame({'Title': list(title_for_np), 'NP': list(np_chunk)}, columns=['Title', 'NP'])
np_df

,Title,NP
0,Arrival,(hope)
1,Arrival,"(good, sci, -, fi, film, recent, time, gunfight)"
2,Arrival,"(melancholy, ,, mysterious, measure, track, st..."
3,Arrival,"(alien, creature)"
4,Arrival,"(Arrival, language, function)"
...,...,...
256,interstellar_2014,"(Christopher, Nolan, good, film, deny, ambition)"
257,interstellar_2014,"(interstellar, grand, space, adventure, worthy..."
258,interstellar_2014,"(,, powerfully, emotional, story, bond, father..."
259,interstellar_2014,"(bold, science, -, fiction, movie)"


In [266]:
np_df['NP']

0                                             (us)
1                                           (hope)
2                   (the, best, sci, -, fi, films)
3                                  (recent, times)
4                                      (gunfights)
                          ...                     
581                                   (that, love)
582                                          (one)
583    (the, boldest, science, -, fiction, movies)
584                                            (I)
585                (narrative, and, visual, terms)
Name: NP, Length: 586, dtype: object

3. Output all the chunks in a single list for each review, and submit that output for this assignment.
Also submit a brief written summary of what you did (describe your selection of genre, your
source of reviews, how many you collected, and by what means).

I selected Sci-fi/Fantasy as a genre, and utilized beautiful soup and Rotten Tomatoes website to collect reviews.I collected 100 reviews, using the first page of reviews overall on the Rotten Tomatoes Website. The dataframe of the reviews also has the url for the full review, which could later be used to pull full text. While there was an API for Rotten Tomatoes, I found that simply scraping the url was quicker with beautiful soup, so I leveraged that instead of the API (examples I found of the API calls online didn't work). It looked like IMDB no longer allowed web scraping, which was why I chose this website.